In [ ]:
# -*- coding: utf-8 -*-
import cv2
import numpy as np
import time
from pymavlink import mavutil
import socket
import threading

# Q.py 코드 시작

# W.py 코드의 연결 문자열과 생성 부분을 추가
connection_string = "/dev/ttyAMA0"
master = mavutil.mavlink_connection(connection_string, baud=57600)
master.wait_heartbeat()

ESC = 1500
SER = 1500

# W.py 코드의 함수들 추가

# Function to set the vehicle mode
def set_mode(mode):
    mode_mapping = {
        'MANUAL': 0,
        'GUIDED': 4,
        'HOLD': 2,
        'AUTO': 3,
        'STEERING': 1,
        'LOITER': 5,
        'RTL': 6,
        'SMART_RTL': 7,
        'GUIDED_NOGPS': 20
    }
    mode_id = mode_mapping.get(mode)
    
    if mode_id is None:
        print("Invalid mode specified")
        return

    master.mav.command_long_send(
        master.target_system,
        master.target_component,
        mavutil.mavlink.MAV_CMD_DO_SET_MODE,
        0,
        mode_id,
        0, 0, 0, 0, 0, 0
    )
# Function to set the servo PWM value-servo
def set_servo(servo_num, pwm_value):
    master.mav.command_long_send(
        master.target_system, master.target_component,
        mavutil.mavlink.MAV_CMD_DO_SET_SERVO,
        0,  # Confirmation
        servo_num, pwm_value, 0, 0, 0, 0, 0
    )
def forward(n):
    set_servo(4,n+85)
    print("forward")
    time.sleep(1)
def backward(n):
    set_servo(4,n-86)
    print("backward")
    time.sleep(2)
def right(n):
    set_servo(6,n-350)
    print("right")
    time.sleep(2)
def left(n):
    set_servo(6,n+300)
    print("left")
    time.sleep(2)
def stop(n):
    print("stop")
    set_servo(6,n)
    set_servo(4,n)
    time.sleep(1)
def Turn_left():
    print("Turn_right")
    stop(ESC)
    right(ESC)
    for i in range(4):
	    forward(ESC)
# Set the mode to MANUAL
set_mode("MANUAL")
time.sleep(1)
#Trun_left()

# Q.py 코드의 line_tracing 함수 및 나머지 코드 부분


count = 0
turn_left_start = None

def line_tracing(frame):
    global count
    global check
    global turn_left_start
    global counting_star
    global BR
    action=""
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    _, threshold = cv2.threshold(gray, 25, 255, cv2.THRESH_BINARY_INV)

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    red_mask = cv2.inRange(hsv, lower_red, upper_red)

    # 화면 가운데 픽셀의 H값 확인
    mid_pixel_h = hsv[height // 2, width // 2, 0]
    red_detected = lower_red[0] <= mid_pixel_h <= upper_red[0]

    contours, hierarchy = cv2.findContours(threshold, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

    cnt = 0
    center_x = 0
    if red_detected:
        print("left")
        check = "RED"
        BR=False
        counting_star+=1
        if count >= 11:  # Add this condition
            count = 1
    else:
        check = "No"
        if BR==False:
            BR=True
            count+=1
        
    for i in range(len(contours)):
        if hierarchy[0][i][3] == -1:
            if cv2.contourArea(contours[i]) > 200:
                (x, y, w, h) = cv2.boundingRect(contours[i])
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                center_x = x + (w // 2)
                center_y = y + (h // 2)
                cnt += 1


    if count % 3 == 0:
        if turn_left_start is None:
            turn_left_start = time.time()
        if time.time() - turn_left_start < 1.5:
            action = "Turn Left"
        else:
            turn_left_start = None
            action="Go"
    else:
        turn_left_start = None
        action="Go"

    return cnt, action
# 빨간색 범위 설정
lower_red = np.array([165, 50, 50])
upper_red = np.array([175, 255, 255])
counting_star=0
cnt = 0
check = "No"
cap = cv2.VideoCapture(0)
BR=True

# C.py 코드 시작
HOST = '192.168.0.3'  # Change this to the server IP if not running on the same machine
PORT = 12345

client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client.connect((HOST, PORT))

def send_count_to_server(count):
    message = f'count:{count}'
    client.send(message.encode('utf-8'))

def receive():
    while True:
        try:
            message = client.recv(1024).decode('utf-8')
            if message == 'NAME':
                client_name = input("Enter your name: ")
                client.send(client_name.encode('utf-8'))
            else:
                print(message)
        except:
            print("An error occurred.")
            client.close()
            break

def main():
    receive_thread = threading.Thread(target=receive)
    receive_thread.start()

    # 코드의 마지막 부분에서 action을 수행하는 부분을 추가
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        height, width = frame.shape[:2]
        cnt, action = line_tracing(frame)

        # A.py에서 반환되는 action에 따라 W.py 코드의 기능 호출
        if action == "Go":
            forward(ESC)
        elif action == "Turn Left":
            Turn_left()
        elif action == "Stop":
            stop(ESC)

        # Send count to the server
        send_count_to_server(count)

        cv2.putText(frame, f"Lines: {cnt}, Red: {check}, Count: {count}, Action: {action}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow('Frame', frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()
